### Server Setup

Creating a stream of data & sending it to the clients.

<br>
<br>

In [1]:
# to specify delay between two consecutive records in stream
DELAY = 0.1

In [2]:
# Importing required modules

# data streaming
import asyncio
import websockets
from time import sleep

# data processing
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
# Read csv files of merged data

# # original data
# data = pd.read_csv("../data/IoT_Garage_Door.csv", low_memory=False)

# test data for presentation purpose
data = pd.read_csv("../data/test_garage_stream.csv", low_memory=False)

In [4]:
# dropping NaN or NA values
data = data.dropna()

# encoding string type variables to numeric type (required for model training)
encoder=LabelEncoder()
data['type']=encoder.fit_transform(data['type'])
data['state']=encoder.fit_transform(data['state'])
data['sphone_signal']=encoder.fit_transform(data['sphone_signal'])

In [5]:
data.head()

,date,time,state,sphone_signal,label,type
0,1-Apr-19,20:53:44,1,1,0,3
1,1-Apr-19,20:53:49,0,0,0,3
2,1-Apr-19,20:53:49,1,1,0,3
3,1-Apr-19,20:53:54,0,0,0,3
4,1-Apr-19,20:53:54,1,1,0,3


In [6]:
async def server(websocket):
    """
    Server function to stream data row-wise in a comma separated string format.
    """

    # convert a row to space(" ") separated string format
    temp = data.to_string(header=False,index=False,
                  index_names=False).split('\n')

    # now convert above row to comma(",") separated string format
    rows = [','.join(ele.split()) for ele in temp]
    
    c=0    # variable to keep track of number of records sent

    # while 'c' is less than & equal to total number of records in dataset
    # this loop is responsible for creating data stream
    while(len(data)>c):
        # send stringified row to the client
        await websocket.send(rows[c])

        # wait of acknowledgement
        ok = await websocket.recv()
        if(bool(ok)):
            c+=1    # if successfully received then increment 'c' by 1
        
        # add artificial delay to simulate a remote server client setup having communication delays due to network traffic
        sleep(DELAY)

async def main():
    async with websockets.serve(server, "localhost", 8765, ping_interval=None):
        await asyncio.Future()  # run forever

await main()

connection handler failed
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/websockets/legacy/server.py", line 232, in handler
    await self.ws_handler(self)
  File "/tmp/ipykernel_29757/618672361.py", line 19, in server
    await websocket.send(rows[c])
  File "/usr/local/lib/python3.10/dist-packages/websockets/legacy/protocol.py", line 620, in send
    await self.ensure_open()
  File "/usr/local/lib/python3.10/dist-packages/websockets/legacy/protocol.py", line 930, in ensure_open
    raise self.connection_closed_exc()
websockets.exceptions.ConnectionClosedOK: sent 1001 (going away); then received 1001 (going away)


CancelledError: 

<br>
<br>
<center><b>End of File</b></center>